# Lecture 9: Softmax Classifier

Consider a situation classifing into n classes.
To make n outputs, we need to use a m x n matrix!
How do we change to n outputs to probability?

## Softmax

$$ \sigma (z)_j = \frac {e^{z_j}}{ \sum_{k=1}^{K} e^{z_k} }  (j =1, 2, ..., K) $$

X -> Linear Model -> Scores (Logits) -> SoftMax -> Probabilities (Choose highest for prediction)

## Cost Function: cross entropy

$$ D(\hat{Y}, Y) = -Ylog\hat{Y} $$
$$ Loss = \frac {1}{N} \sum_ D(s(wx_i +b) (= \hat{y}), y_i) $$

## Implementation using PyTorch

When using CrossEntropyLoss, Y_pred are logits(Not Softmax), and Y are Class (Not One-hot encoded Values.)

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
#from torch.utils.data import Dataset, DataLoader

# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss =nn.CrossEntropyLoss()

# target is of size nBatch
# each Element in target has to have 0 <= value <= nClasses (0-2)
Y = Variable(torch.LongTensor([2,0,1]), requires_grad = False)

#input is of size nBatch x nClasses = 1x4
# Y_pred are logits (not softmax)
Y_pred1= Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                [1.1, 0.1, 0.2],
                                [0.2, 2.1, 0.1]]))
Y_pred2= Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                [0.2,0.3,0.5],
                                [0.2,0.2,0.5]]))

l1 = loss(Y_pred1, Y)
l2= loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data,
    "\nPyTorch Loss2=", l2.data)

PyTorch Loss1 =  tensor(0.4966) 
PyTorch Loss2= tensor(1.2389)


## Use Softmax Classifier on MNIST

## Model Design

Input : 28 * 28 pixel image data
Input Layer (784) -> Hidden Layers -> Output Layer (10)

## Example


In [9]:
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.autograd import Variable


# Training Settings
batch_size=64
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())
# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


# Model Design
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1=nn.Linear(784,520)
        self.l2=nn.Linear(520,320)
        self.l3=nn.Linear(320,240)
        self.l4=nn.Linear(240,120)
        self.l5=nn.Linear(120,10)

    def forward(self, x):
        # Flaten the data (n,1,28,28) -> (n,784)
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) #No need activation

model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-porbability
        pred = torch.max(output.data, 1)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.301126
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.310943
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.310502
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.312083
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.300453
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.299345
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.301799
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.299917
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.307039
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.302605
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.300593
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.294588
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.301765
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.302658
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.301330
Train Epoch: 1 | Batch Status: 9600/60

/var/folders/j6/9s4ycw_d3q567rb82x3f01l40000gn/T/ipykernel_2864/529761606.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)


Test set: Average loss: 0.0234, Accuracy: 5420/10000 (54%)
Train Epoch: 2 | Batch Status: 0/60000 (0%) | Loss: 1.518718
Train Epoch: 2 | Batch Status: 640/60000 (1%) | Loss: 1.396291
Train Epoch: 2 | Batch Status: 1280/60000 (2%) | Loss: 1.421184
Train Epoch: 2 | Batch Status: 1920/60000 (3%) | Loss: 1.447849
Train Epoch: 2 | Batch Status: 2560/60000 (4%) | Loss: 1.606114
Train Epoch: 2 | Batch Status: 3200/60000 (5%) | Loss: 1.279867
Train Epoch: 2 | Batch Status: 3840/60000 (6%) | Loss: 1.189844
Train Epoch: 2 | Batch Status: 4480/60000 (7%) | Loss: 1.474188
Train Epoch: 2 | Batch Status: 5120/60000 (9%) | Loss: 1.390773
Train Epoch: 2 | Batch Status: 5760/60000 (10%) | Loss: 1.167181
Train Epoch: 2 | Batch Status: 6400/60000 (11%) | Loss: 1.178540
Train Epoch: 2 | Batch Status: 7040/60000 (12%) | Loss: 1.192511
Train Epoch: 2 | Batch Status: 7680/60000 (13%) | Loss: 1.111317
Train Epoch: 2 | Batch Status: 8320/60000 (14%) | Loss: 1.171095
Train Epoch: 2 | Batch Status: 8960/60000 (1